# Пример записи и чтения из Kafka

## Пример с использованием Spark

### Запись в Kafka

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, LongType, StringType, IntegerType, DateType, TimestampType, FloatType
from pyspark.sql.functions import col, cast, date_trunc, sum, dayofweek, hour, dayofmonth, lit, from_json, to_json

import os

In [ ]:
spark = SparkSession.builder.appName('kafka').getOrCreate()

In [ ]:
# формирование dataframe
values= [
        (4,4.0,'4'),
        (5,5.0,'5'),
        (6,6.0,'6')
    ]

schemaData = StructType([ \
    StructField("c1",IntegerType(),True), 
    StructField("c2",FloatType(),True), 
    StructField("c3",StringType(),True)
  ])

df = spark.createDataFrame(values,schema=schemaData)

In [ ]:
# посмотрим данные
df.show()

In [ ]:
# имя топика
topicName = 'topic1'

In [ ]:
# параметры подключения
KAFKA_SERVERS = os.environ['KAFKA_SERVERS']
print(KAFKA_SERVERS)

In [ ]:
# запись в топик
df.selectExpr('to_json(struct(*)) as value').write.format("kafka"). \
    option("kafka.bootstrap.servers", KAFKA_SERVERS). \
    option("topic", topicName).save()

In [ ]:
spark.stop()

### Чтение из Kafka

In [ ]:
spark = SparkSession.builder.appName('kafka').getOrCreate()

In [ ]:
# чтение из кафки и транформация в исходный вид
df2 = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_SERVERS) \
  .option("subscribe", topicName) \
  .load()
df3 = df2.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)"). \
          withColumn("value",from_json(col("value"),schemaData)). \
          select(col("value.*"))

df3.show()

In [ ]:
spark.stop()